# Notebook 00: Configuration de l'Environnement

Bienvenue dans le projet "Cognitive Swarm: Multi-Agent Knowledge Discovery Engine" !

Ce notebook vous guidera à travers les étapes initiales de configuration de votre environnement de développement.

## Étapes Préalables

1.  **Installer Conda/Miniconda/Anaconda** : Si vous ne l'avez pas déjà, installez une distribution Conda adaptée à votre système d'exploitation depuis [Anaconda.com](https://www.anaconda.com/products/distribution) ou [Miniconda](https://docs.conda.io/en/latest/miniconda.html).

2.  **Créer l'Environnement Conda** :
    * Ouvrez un terminal ou une invite de commande Anaconda.
    * Naviguez jusqu'à la racine de ce projet (`cognitive-swarm-agents/`).
    * Exécutez la commande suivante pour créer l'environnement nommé `cognitive-swarm` à partir du fichier `environment.yml` fourni :
        ```bash
        conda env create -f environment.yml
        ```
    * Si l'environnement existe déjà et que vous souhaitez le mettre à jour :
        ```bash
        conda env update -f environment.yml --prune
        ```

3.  **Activer l'Environnement Conda** :
    ```bash
    conda activate cognitive-swarm
    ```
    Vous devriez voir `(cognitive-swarm)` au début de votre invite de commande.

4.  **Créer le Fichier `.env`** :
    * À la racine du projet (`cognitive-swarm-agents/`), créez un fichier nommé `.env`.
    * Copiez le contenu de `.env.example` dans votre nouveau fichier `.env`.
    * Remplissez les valeurs des variables d'environnement requises dans votre fichier `.env`, notamment :
        * `OPENAI_API_KEY`
        * `MONGO_URI` (votre chaîne de connexion MongoDB Atlas)
        * `WANDB_API_KEY` (si vous utilisez Weights & Biases)
        * Optionnellement, d'autres clés API (`ANTHROPIC_API_KEY`, `GROQ_API_KEY`, `TAVILY_API_KEY`) si vous prévoyez d'utiliser ces services.

## Vérification de l'Environnement

Les cellules de code ci-dessous vous aideront à vérifier que tout est correctement configuré.

In [ ]:
# Importer les bibliothèques nécessaires pour ce notebook de configuration
import os
from dotenv import load_dotenv
from pathlib import Path
import importlib

# Tentative d'installation de python-dotenv si non présent (devrait être dans environment.yml)
try:
    importlib.import_module('dotenv')
except ImportError:
    print("Installation de python-dotenv...")
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "python-dotenv"])
    print("python-dotenv installé. Veuillez relancer ce kernel Jupyter.")
    # sys.exit() # Ou demander à l'utilisateur de relancer

print("Imports de base réussis.")

### 1. Chargement des Variables d'Environnement

Cette cellule charge les variables à partir de votre fichier `.env` et affiche certaines d'entre elles pour vérification (les clés API ne seront pas affichées intégralement).

In [ ]:
# Charger les variables du fichier .env situé à la racine du projet
# Le notebook est dans notebooks/, donc ../.env
dotenv_path = Path().resolve().parent / ".env" # Path().resolve() donne le répertoire du notebook

if dotenv_path.exists():
    load_dotenv(dotenv_path=dotenv_path)
    print(f"Variables d'environnement chargées depuis : {dotenv_path}")
else:
    print(f"Fichier .env non trouvé à l'emplacement attendu : {dotenv_path}")
    print("Veuillez vous assurer d'avoir créé et configuré votre fichier .env à la racine du projet.")

# Importer les settings de notre projet (qui utilisent aussi dotenv)
# Pour que cela fonctionne, assurez-vous que le PYTHONPATH est correct
# ou que vous exécutez Jupyter depuis la racine du projet.
# Si exécuté depuis le dossier notebooks :
import sys
project_root = Path().resolve().parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

try:
    from config.settings import settings
    print("Configuration du projet (settings.py) chargée avec succès.")
except ImportError as e:
    print(f"Erreur lors de l'import de config.settings: {e}")
    print("Assurez-vous que vous exécutez ce notebook depuis le répertoire 'notebooks/' ou que la racine du projet est dans PYTHONPATH.")
    settings = None # Pour éviter des erreurs plus loin si l'import échoue

if settings:
    print(f"\n--- Vérification des Settings Chargés ---")
    print(f"Nom du Projet: {settings.PROJECT_NAME}")
    print(f"Mode Debug: {settings.DEBUG}")
    print(f"Environnement Python (PYTHON_ENV): {settings.PYTHON_ENV}")

    print(f"\n--- Configuration des LLMs Génératifs ---")
    print(f"Fournisseur LLM Génératif par Défaut: {settings.DEFAULT_LLM_MODEL_PROVIDER}")
    print(f"  Si OpenAI: Modèle = {settings.DEFAULT_OPENAI_GENERATIVE_MODEL}")
    print(f"  Si HuggingFace API: Repo ID = {settings.HUGGINGFACE_REPO_ID}")
    print(f"  Si Ollama: Modèle Génératif = {settings.OLLAMA_GENERATIVE_MODEL_NAME}, Base URL = {settings.OLLAMA_BASE_URL}")
    
    print(f"\n--- Configuration des Embeddings ---")
    print(f"Fournisseur d'Embedding par Défaut: {settings.DEFAULT_EMBEDDING_PROVIDER}")
    print(f"  Pour OpenAI: Modèle = {settings.OPENAI_EMBEDDING_MODEL_NAME}, Dimension = {settings.OPENAI_EMBEDDING_DIMENSION}")
    print(f"  Pour HuggingFace: Modèle = {settings.HUGGINGFACE_EMBEDDING_MODEL_NAME}, Dimension = {settings.HUGGINGFACE_EMBEDDING_MODEL_DIMENSION}")
    print(f"  Pour Ollama: Modèle = {settings.OLLAMA_EMBEDDING_MODEL_NAME}, Dimension = {settings.OLLAMA_EMBEDDING_MODEL_DIMENSION}, Base URL = {settings.OLLAMA_BASE_URL}")

    print(f"\n--- Clés API (vérification de présence) ---")
    print(f"  OPENAI_API_KEY chargée: {bool(settings.OPENAI_API_KEY)}")
    if settings.OPENAI_API_KEY:
        print(f"    (début : {settings.OPENAI_API_KEY[:4]}...)")
    print(f"  HUGGINGFACE_API_KEY chargée: {bool(settings.HUGGINGFACE_API_KEY)}")
    if settings.HUGGINGFACE_API_KEY:
        print(f"    (début : {settings.HUGGINGFACE_API_KEY[:6]}...)")
    print(f"  WANDB_API_KEY chargée: {bool(settings.WANDB_API_KEY)}")
    if settings.WANDB_API_KEY:
        print(f"    (début : {settings.WANDB_API_KEY[:4]}...)")
    print(f"  ANTHROPIC_API_KEY chargée: {bool(settings.ANTHROPIC_API_KEY)}")
    print(f"  GROQ_API_KEY chargée: {bool(settings.GROQ_API_KEY)}")
    print(f"  TAVILY_API_KEY chargée: {bool(settings.TAVILY_API_KEY)}")

    print(f"\n--- Configuration MongoDB ---")
    print(f"  MONGO_URI: {settings.MONGO_URI[:20]}... (tronqué pour affichage)")
    print(f"  MONGO_DATABASE_NAME: {settings.MONGO_DATABASE_NAME}")
    print(f"  LANGGRAPH_CHECKPOINTS_COLLECTION: {settings.LANGGRAPH_CHECKPOINTS_COLLECTION}")


    print(f"\n--- Configuration des Données et Chemins ---")
    print(f"  DATA_DIR: {settings.DATA_DIR}")
    print(f"  Chemin du dataset d'évaluation RAG (par défaut): {settings.EVALUATION_DATASET_PATH}")

    if settings.DATA_DIR.exists():
        print(f"  Le répertoire DATA_DIR ({settings.DATA_DIR}) existe.")
    else:
        print(f"  ATTENTION : Le répertoire DATA_DIR ({settings.DATA_DIR}) n'existe pas. Certains scripts pourraient échouer.")
        print(f"  Vous pourriez avoir besoin de le créer manuellement ou il sera créé par run_ingestion.py (pour ses sous-dossiers).")
    
    # Vérifier le dataset d'évaluation si le chemin est défini
    if settings.EVALUATION_DATASET_PATH:
        eval_path = Path(settings.EVALUATION_DATASET_PATH)
        if eval_path.is_file(): # Vérifier si c'est un fichier
            print(f"  Le fichier de dataset d'évaluation RAG ({eval_path}) existe.")
        else:
            print(f"  ATTENTION : Le fichier de dataset d'évaluation RAG ({eval_path}) n'existe PAS ou n'est pas un fichier.")
    else:
        print("  Aucun chemin de dataset d'évaluation RAG n'est configuré dans settings.py (EVALUATION_DATASET_PATH est None).")

else:
    print("\nLes settings du projet n'ont pas pu être chargés. Veuillez vérifier l'import et le fichier .env.")

### 2. Test de Connexion à MongoDB

Cette cellule tente de se connecter à votre instance MongoDB en utilisant l'URI fournie dans `.env` et chargée via `settings.py`.
Assurez-vous que votre base de données MongoDB Atlas est accessible (IP autorisée, etc.) ou que votre instance locale est en cours d'exécution.

In [ ]:
libraries_to_check = [
    "langchain",
    "langchain_core",
    "langchain_openai",
    "langchain_community", # Ajouté pour être explicite, car HuggingFaceEmbeddings et OllamaEmbeddings en viennent
    "langgraph",
    "llama_index.core",
    "llama_index.vector_stores.mongodb",
    "llama_index.embeddings.openai",
    "llama_index.embeddings.huggingface", # <<< NOUVEAU CHECK
    "llama_index.embeddings.ollama",      # <<< NOUVEAU CHECK
    "pymongo",
    "motor",
    "arxiv",
    "fitz", # PyMuPDF
    "tiktoken",
    "pandas",
    "wandb",
    "crewai", # Ajouté pour être explicite
    "pydantic",
    "pydantic_settings",
    "fastapi", # Ajouté pour être explicite
    "uvicorn"  # Ajouté pour être explicite
]

print("--- Vérification de l'import des bibliothèques principales ---")
all_libraries_found = True
for lib_name in libraries_to_check:
    try:
        importlib.import_module(lib_name)
        print(f"✅ {lib_name} : Trouvé et importable.")
    except ImportError:
        print(f"❌ {lib_name} : NON TROUVÉ. Veuillez vérifier votre environnement Conda ('environment.yml') et vos requirements pip.")
        all_libraries_found = False

if all_libraries_found:
    print("\n🎉 Toutes les bibliothèques principales listées semblent être correctement installées !")
else:
    print("\n⚠️ Certaines bibliothèques principales listées sont manquantes. Veuillez vérifier l'installation de votre environnement Conda/pip.")

### 3. Vérification des Bibliothèques Principales

Cette cellule tente d'importer les bibliothèques majeures utilisées dans le projet pour s'assurer qu'elles sont bien installées dans l'environnement `cognitive-swarm`.

In [ ]:
libraries_to_check = [
    "langchain",
    "langchain_core",
    "langchain_openai",
    "langgraph",
    "llama_index.core",
    "llama_index.vector_stores.mongodb",
    "llama_index.embeddings.openai",
    "pymongo",
    "motor",
    "arxiv",
    "fitz", # PyMuPDF
    "tiktoken",
    "pandas",
    "wandb",
    "pydantic",
    "pydantic_settings"
]

print("--- Vérification de l'import des bibliothèques principales ---")
all_libraries_found = True
for lib_name in libraries_to_check:
    try:
        importlib.import_module(lib_name)
        print(f"✅ {lib_name} : Trouvé et importable.")
    except ImportError:
        print(f"❌ {lib_name} : NON TROUVÉ. Veuillez vérifier votre environnement Conda ('environment.yml').")
        all_libraries_found = False

if all_libraries_found:
    print("\n🎉 Toutes les bibliothèques principales semblent être correctement installées !")
else:
    print("\n⚠️ Certaines bibliothèques principales sont manquantes. Veuillez vérifier l'installation de votre environnement Conda.")

## Configuration Terminée

Si toutes les cellules ci-dessus se sont exécutées sans erreur majeure (en particulier les erreurs d'import ou de connexion), votre environnement est probablement prêt pour la suite du projet "Cognitive Swarm".

Vous pouvez maintenant passer aux notebooks suivants pour explorer les différentes parties du système.